# Primality Testing

For RSA, we needed two large primes $p$ and $q$, to compute $n$ and $\phi(n)$. But how do we find these large primes? In practice, the most efficient way to find large random primes is to generate large random numbers and test them to see if they are prime. This brings us to <i>primality testing</i>, a family of algorithms that (deterministicly or probabilisticly) tell if a number is prime or composite.

In [6]:
import math
import numpy as np

In [45]:
def quadratic_residue(a, p):
    """
    Determine whether a is a quadratic residue modulo p
    
    p is prime
    a is an integer
    
    return True if a^((p-1)/2) = 1 (mod p)
    """
    
    base = (a ** ((p - 1) / 2)) % p
    
    return base == 1

In [47]:
def legendre(a, p):
    """
    p must be prime
    a is an integer
    """
    
    m = a % p
    
    if m == 0:
        return 0
    
    if quadratic_residue(a, p):
        return 1
    
    return -1

In [48]:
def is_prime(n):
    """
    If a number n is composite, it must have a prime 
        divisor between 2 and sqrt(n) inclusive
        Proof: Suppose n = pq is composite, where p,q
            are prime, and p,q > sqrt(n).
            Then pq > sqrt(n)*sqrt(n) = n, thus n!= pq
    We loop through all prime numbers from 2 to sqrt(n),
        seeing if n % p == 0. If so, then p|n, and n is 
        composite. If we reach the end of the loop,
        n is determined to be prime
    """

    if n < 10:
        if n in [2, 3, 5, 7]:
            return True
        return False
    upper_limit = math.ceil(math.sqrt(n))
    for i in range(2, upper_limit + 1):
        if is_prime(i):
            if n%i == 0:
                return False
    
    # At this point we have looped through all possible prime
    #  divisors, and none divide n, so n is not prime
    
    return True

In [49]:
def prime_factorization(n, prime_list = None):
    """
    Returns the prime factorization of n
    Input:
        n - integer
        
    Output:
        prime_decomp - list of tuples (p_i, a_i) where n = prod(p_i ^ a_i)
    """
    
    upper_limit = math.ceil(math.sqrt(n))
    prime_decomp = list()
    
    if is_prime(n):
        prime_decomp.append((n, 1))
        
        return prime_decomp
    
    if prime_list:
        for p in prime_list:
            a = 0
            while n % p == 0:
                n = n / p
                a += 1
            if a > 0:
                prime_decomp.append((p, a))
                
    for p in range(2, n):
        if n == 1:
            
            return prime_decomp
        
        if is_prime(p):
            a = 0
            while n % p == 0:
                n = n / p
                a += 1
            if a > 0:
                prime_decomp.append((p, a))
                
    return prime_decomp

In [69]:
def jacobi(a, n, verbose=False):
    """
    Calculate the Jacobi symbol of a on n
    
    Since this involves calculating a^((n-1)/2) mod n
        which can be inefficient for large a, n, we 
        include some steps for simplification prior to 
        computation
    """
    if verbose:
        print(f'computing jacobi({a},{n})')
    if n % 2 == 1:
        if verbose:
            print('n odd')
        if a == 2:
            if verbose:
                print('a = 2')
            if np.abs(n % 8) == 1:
                return 1
            if np.abs(n % 8) == 3:
                return -1
        if a > n:
            if verbose:
                print('a greater than n, reducing a modulo n')
            # If a > n then 
            #   jacobi(a,n) = jacobi(a%n, n)
            a = a % n
            return jacobi(a, n, verbose)
        
        if a % 2 == 1:
            if verbose:
                print('a odd')
            # If a is also odd then
            #  jacobi(a,n)=-1*jacobi(n, a) if
            #  a,n = 3 mod 4, and 
            #  jacobi(a,n)=jacobi(n, a) otherwise
            
            # Since we have passed the previous if
            #  statement, we have a <= n
            if a % 4 == 3 and n % 4 == 3:
                return -1 * jacobi(n, a, verbose)
            else:
                return jacobi(n, a, verbose)
    
    pd = prime_factorization(a)
    
    jacobi_result = int(np.prod([jacobi(x[0], n) ** x[1] for x in pd]))
    
    return jacobi_result

In [70]:
def solovay_strassen(n):
    """
    Solovay-Strassen primality test
    Input:
        n - integer to be tested for primality
    Output:
        is_prime - True if n is prime, False otherwise
    """
    is_prime = False
    
    a = np.random.randint(1, n)
    x = jacobi(a, n)
    
    if x == 0:
        return is_prime
    
    y = a ** ((n - 1) / 2) % n
    if  x % n == y:
        is_prime = True
    
    return is_prime

In [71]:
jacobi(7411, 9283)

-1